# Exercices

## **Préliminaires**: Clone de votre repo et imports

In [1]:
! git clone https://github.com/nanopiero/exam_2025.git
! cp exam_2025/utils/utils_exercices.py .

import copy
import numpy as np
import torch

Cloning into 'exam_2025'...
remote: Enumerating objects: 100, done.
remote: Counting objects: 100% (22/22), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 100 (delta 12), reused 7 (delta 7), pack-reused 78 (from 2)
Receiving objects: 100% (100/100), 3.16 MiB | 11.08 MiB/s, done.
Resolving deltas: 100% (33/33), done.


**Clef personnelle pour la partie théorique**

Dans la cellule suivante, choisir un entier entre 100 et 1000 (il doit être personnel). Cet entier servira de graine au générateur de nombres aléatoire a conserver pour tous les exercices.



In [3]:
mySeed = 278

**Remarque** : il est important de respecter la consigne : un choix de graine réellement aléatoire ne doit pas déboucher sur des paires de graines identiques dans une promotions. Les paires identiques sont naturellement suspectées.

\

---

\

\

**Exercice 1** *Une relation linéaire*

La fonction *generate_dataset* fournit deux jeux de données (entraînement et test). Pour chaque jeu de données, la clef 'inputs' donne accès à un tableau numpy (numpy array) de prédicteurs empilés horizontalement : chaque ligne $i$ contient trois prédicteurs $x_i$, $y_i$ et $z_i$. La clef 'targets' renvoie le vecteur des cibles $t_i$. \

Les cibles sont liées aux prédicteurs par le modèle:
$$ t = \theta_0 + \theta_1 x + \theta_2 y + \theta_3 z + \epsilon$$ où $\epsilon \sim \mathcal{N}(0,\eta)$


In [ ]:
from utils_exercices import generate_dataset, Dataset1
train_set, test_set = generate_dataset(mySeed)

In [22]:
test_set['targets'].shape

(500,)

**Q1** Par quelle méthode simple peut-on estimer les coefficients $\theta_k$ ? La mettre en oeuvre avec la librairie python de votre choix.

On peut utiliser une régression linéaire, par exemple en utilisant sklearn:

**Prompt**: "A training set and a testing set are stored in python dictionnaries. The train_set dictinnary contain:
train_set['inputs']:  horizontally stacked predictands in the form of a numpy array of shape 1000 x 3. and train_set['targets'] : array hor. stacked targets of shape 1000 x 1.
The test set only contain 500 lines for both inputs and targets.
Please provide a snippet that sole the ML problem by linear regression. Provide estimated coefficients and give MSE on the test set."

In [23]:
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Assume train_set and test_set dictionaries are already defined
# Extract inputs and targets
X_train = train_set['inputs']  # Shape (1000, 3)
y_train = train_set['targets']  # Shape (1000, 1)
X_test = test_set['inputs']  # Shape (500, 3)
y_test = test_set['targets']  # Shape (500, 1)

# Create and train the linear regression model
model = LinearRegression()
model.fit(X_train, y_train)

# Get estimated coefficients
coefficients = model.coef_
intercept = model.intercept_
print("Estimated coefficients:", coefficients)
print("Intercept:", intercept)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Compute Mean Squared Error
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error on test set:", mse)


Estimated coefficients: [2.73396719 2.9265279  5.584744  ]
Intercept: 13.905807493577722
Mean Squared Error on test set: 3.5792728778129765


**Q2** Dans les cellules suivantes, on se propose d'estimer les $\theta_k$ grâce à un réseau de neurones entraîné par SGD. Quelle architecture s'y prête ? Justifier en termes d'expressivité et de performances en généralisation puis la coder dans la cellule suivante.

Un perceptron à une couche est suffisant pour exprimer la relation  linéaire entre entrées et cibles. Pour éviter tout surapprentissage, nous prenons pas plus de paramètres que nécessaire (quatre en comptant le biais).

prompt:
"please complete these following snippets to train a one layer perceptron with 4 weights (including a bias) through a standard SGD. Provide the weights after the training phase and test the perceptron over the test set. [...]"

In [42]:
# Initialize model, loss, and optimizer
mySimpleNet = SimpleNet()
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(mySimpleNet.parameters(), lr=0.01)

**Q3** Entraîner cette architecture à la tâche de régression définie par les entrées et sorties du jeu d'entraînement (compléter la cellule ci-dessous).

In [43]:
# Training loop
num_epochs = 500
for epoch in range(num_epochs):
    for batch_inputs, batch_targets in dataloader:
        # Forward pass
        outputs = mySimpleNet(batch_inputs)
         # attention : le unsqueeze est nécessaire. Une IA
         # ne le fournira pas systématiquement (elle n'a pas compris
         # a priori la forme du tenseur batch_targets)
        loss = criterion(outputs, batch_targets.unsqueeze(1))

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

**Q4** Où sont alors stockées les estimations des  $\theta_k$ ? Les extraire du réseau *mySimpleNet* dans la cellule suivante.

In [45]:
# Get trained weights
trained_weights = mySimpleNet.linear.weight.data
trained_bias = mySimpleNet.linear.bias.data
print("Trained Weights:", trained_weights.numpy())
print("Trained Bias:", trained_bias.numpy())

Trained Weights: [[2.7327583 2.9275002 5.58484  ]]
Trained Bias: [13.905466]


**Q5** Tester ces estimations sur le jeu de test et comparer avec celles de la question 1. Commentez.

In [48]:
# Test the model on test set
test_inputs = torch.tensor(test_set['inputs'], dtype=torch.float32)
test_targets = torch.tensor(test_set['targets'], dtype=torch.float32)
test_outputs = mySimpleNet(test_inputs)

# Même remarque qu'au dessus:
test_loss = criterion(test_outputs, test_targets.unsqueeze(1)).item()
print("Test MSE:", test_loss)


Test MSE: 3.5797951221466064


On a logiquement trouvé des coefficients très légèrement moins bons par SGD que par la méthode "exacte" mise en oeuvre (OLS) avec la bibliothèque sklearn.

\

---

\

**Exercice 2** *Champ réceptif et prédiction causale*

Le réseau défini dans la cellule suivante est utilisé pour faire le lien entre les valeurs $(x_{t' \leq t})$ d'une série temporelle d'entrée et la valeur présente $y_t$ d'une série temporelle cible.

In [4]:
import torch.nn as nn
import torch.nn.functional as F
from utils_exercices import Outconv, Double_conv_causal, Up_causal, Down_causal

class Double_conv_causal(nn.Module):
    '''(conv => BN => ReLU) * 2, with causal convolutions that preserve input size'''
    def __init__(self, in_ch, out_ch, kernel_size=3, dilation=1):
        super(Double_conv_causal, self).__init__()
        self.kernel_size = kernel_size
        self.dilation = dilation
        self.conv1 = nn.Conv1d(in_ch, out_ch, kernel_size=kernel_size, padding=0, dilation=dilation)
        self.bn1 = nn.BatchNorm1d(out_ch)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv1d(out_ch, out_ch, kernel_size=kernel_size, padding=0, dilation=dilation)
        self.bn2 = nn.BatchNorm1d(out_ch)

    def forward(self, x):
        # Apply causal padding manually for the first convolution
        x = F.pad(x, ((self.kernel_size - 1) * self.dilation, 0))  # Pad 2 on the left (time dimension), 0 on the right
        x = self.conv1(x)
        # testx
        x = self.bn1(x)
        x = self.relu(x)

        x = F.pad(x, ((self.kernel_size - 1) * self.dilation, 0))
        x = self.conv2(x)
        # testx
        x = self.bn2(x)
        x = self.relu(x)
        return x


class causalFCN(nn.Module):
    def __init__(self, dilation=1):
        super(causalFCN, self).__init__()
        size = 64
        n_channels = 1
        n_classes = 1
        self.inc = Double_conv_causal(n_channels, size)
        self.down1 = Down_causal(size, 2*size)
        self.down2 = Down_causal(2*size, 4*size)
        self.down3 = Down_causal(4*size, 8*size, pooling_kernel_size=5, pooling_stride=5)
        self.down4 = Down_causal(8*size, 4*size, pooling=False, dilation=2)
        self.up2 = Up_causal(4*size, 2*size, kernel_size=5, stride=5)
        self.up3 = Up_causal(2*size, size)
        self.up4 = Up_causal(size, size)
        self.outc = Outconv(size, n_classes)
        self.n_classes = n_classes

    def forward(self, x):
        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.down4(x4)
        x = self.up2(x5, x3)
        x = self.up3(x, x2)
        x = self.up4(x, x1)
        x = self.outc(x)
        return x

# Exemple d'utilisation
model = causalFCN()
# Série temporelle d'entrée (x_t):
input_tensor1 = torch.rand(1, 1, 10000)
# Série temporelle en sortie f(x_t):
output = model(input_tensor1)
print(output.shape)

torch.Size([1, 1, 10000])


**Q1** De quel type de réseau de neurones s'agit-il ? Combien de paramètres la couche self.Down1 compte-t-elle (à faire à la main) ?
Combien de paramètres le réseau entier compte-t-il (avec un peu de code) ?

Il s'agit d'un FCN (Fully Convolutional Network).

Pour la suite :

**prompt**: " From the following snippet, could you manually count the number of weights in self.Down1 ? Could you provide a snippet that counts the total number of trainable weights ? Could you also present the first answer (manual count) directly in markdown in order to ease the reading of my notebook ?[...]"

**Remarque**: en plus du contenu de la première cellule de l'exercice, j'ai dû ajouter les définitions de Down_causal et Double_conv_causal dans [...] situées dans utils_exercice (sinon, et l'IA le précise normalement, il y a ambiguïté).

Manually Counting the Number of Weights in self.down1

In the causalFCN class, self.down1 is an instance of Down_causal(64, 128). From the Down_causal definition:

    If pooling is enabled (which it is by default), Down_causal consists of:
        A MaxPool1d layer (which has no trainable parameters).
        A Double_conv_causal layer.

Now, looking at Double_conv_causal(64, 128), it contains:

    First Conv1d layer (conv1)
        Input channels: 64
        Output channels: 128
        Kernel size: 3
        Trainable parameters: 128×64×3=24,576128×64×3=24,576
        Bias parameters: 128128
        Total: 24,704

    BatchNorm layer (bn1)
        Trainable parameters: 128128 (scaling factor gamma and bias beta).
        Total: 128

    Second Conv1d layer (conv2)
        Input channels: 128
        Output channels: 128
        Kernel size: 3
        Trainable parameters: 128×128×3=49,152128×128×3=49,152
        Bias parameters: 128128
        Total: 49,280

    BatchNorm layer (bn2)
        Trainable parameters: 128128
        Total: 128

Total Number of Trainable Parameters in self.down1
24,704+128+49,280+128=74,240


In [5]:
import torch

# Instantiate the model
model = causalFCN()

# Count total trainable parameters
total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"Total trainable parameters: {total_params}")

Total trainable parameters: 2872641


**Q2** Par quels mécanismes la taille du vecteur d'entrée est-elle réduite ? Comment est-elle restituée dans la deuxième partie du réseau ?

Réduction de la dimension (spatiale):
- utilisation de maxpool1d (Down_causal)
  avec noyau de taille 2 ou 5.
- paramètre dilation égal à 2.
La dimension est restituée par utilisation de couches convtranspose1d (Up_causal).


**Remarque**: vous pouviez demander ces mécanismes à une IA. Mais limitez la réponse aux éléments demandés (en copiant collant tout le contenu généré par une IA, vous perdez généralement en concision, et vous êtes alors pénalisés).

**Q3** Par quels mécanismes le champ réceptif est-il augmenté ? Préciser par un calcul la taille du champ réceptif en sortie de *self.inc*.

Le champ réceptif est augmenté par l'effet des convolutions, du maxpooling, du paramètre dilation. Mais les skip connections n'ont pas d'effet, contrairement à ce qui est souvent écrit.

Le champ réceptif en sortie de self.inc est de taille cinq (deux convolutions successives de taille avec noyaux de taille trois).

**Remarque** : Là encore, les IA donnaient beaucoup plus de détail que nécessaire. Encore une fois, soyez concis.

**Q4** Par un bout de code, déterminer empiriquement la taille du champ réceptif associé à la composante $y_{5000}$ du vecteur de sortie. \\
Indices:  
- considérer les sorties associées à deux inputs qui ne diffèrent que par une composante...
- attention à la mise à jour de paramètres cachés (voire par exemple [track_running_stats](https://pytorch.org/docs/stable/generated/torch.nn.BatchNorm2d.html)) en mode *.train()*.  

**Notes**:
- J'ai ajouté le deuxième indice après la première session de 2025 : seul un étudiant a pensé à passer en mode "eval" pour éviter les effets des couches de normalisation lors d'évaluation successives.
- J'ai d'abord passé la boucle suivante avec un pas de 100 pour réduire l'intervalle de recherche à 4700 - 5100.

In [55]:
input_tensor1.shape

torch.Size([1, 1, 10000])

In [7]:
import torch

# Set device to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Exemple d'utilisation
model = causalFCN().to(device)
model.eval()  # Set model to evaluation mode

# Série temporelle d'entrée (x_t):
input_tensor1 = torch.rand(1, 1, 10000, device=device)

# Série temporelle en sortie f(x_t):
output = model(input_tensor1)

# Measure receptive field empirically
output_index = 4999
start_index = None
stop_index = None

# for i in range(0,10000,100):
with torch.no_grad():
  for i in list(range(4600,4700,1)) + list(range(4999,5100,1)):
      perturbed_input = input_tensor1.clone()
      perturbed_input[0, 0, i] += 10000. # grosse perturbation
      perturbed_output = model(perturbed_input)
      dist = torch.abs(perturbed_output[0, 0, output_index] - output[0, 0, output_index])
      if dist > 0:
          if start_index is None:
              start_index = i
          stop_index = i

print(f"Receptive field starts at index: {start_index}")
print(f"Receptive field stops at index: {stop_index}")
print(f"Receptive field size: {stop_index - start_index + 1}")

Receptive field starts at index: 4692
Receptive field stops at index: 4999
Receptive field size: 308


**Q5** $y_{5000}$ dépend-elle des composantes $x_{t, \space t > 5000}$ ? Justifier de manière empirique puis préciser la partie du code de Double_conv_causal qui garantit cette propriété de "causalité" en justifiant.  



Dans la fenêtre de code précédente, on a montré la causalité vis à vis de la 5000 ème composante du vecteur de sortie : les composantes suivantes n'influencent pas sa valeur.

Ce qui assure effectivement cette causalité, c'est le padding à gauche avant convolution. On a ajouté autant de zéro à la série **x** qu'il faut pour qu'en sortie, la composante **self.conv1(x)[i]** ne dépende que de **x[i-2:i]**. Même chose pour self.conv2.

**Remarque**: A cause des maxpooling1D, la causalité n'est pas parfaite dans le sens où elle n'est complètement satisfaite que pour les 5000ème, 5020ème, 5040ème ... 4980ème, 4960ème composantes. Un contrexemple (correct) à la causalité suffisait ici à obtenir tous les points.  


\

---

\

\

Exercice 3: "Ranknet loss"

Un [article récent](https://https://arxiv.org/abs/2403.14144) revient sur les progrès en matière de learning to rank. En voilà un extrait :


<img src="https://raw.githubusercontent.com/nanopiero/exam_2025/refs/heads/main/utils/png_exercice3.PNG" alt="extrait d'un article" width="800">

**Q1** Qu'est-ce que les auteurs appellent "positive samples" et "negative samples" ? Donner un exemple.

Dans ce contexte de "learning to rank" où un apprentissage sur des paires comparées ("pairwise comparisons") est mis en oeuvre, l'expression "positive sample" (resp. "negative sample") fait référence à l'éléments de rang supérieur (resp. inférieur) de la paire. Un exemple est donné dans le TP n°7: une paire est constitué de deux images contenant chacune un disque comparées sur le critère de l'intensité du disque.

**Q2** Dans l'expression de $\mathcal{L}_{RankNet}$, d'où proviennent les $z_i$ ? Que représentent-ils ?  

Les $z_i$ sont les sorties du modèle associées aux éléments d'une paire. Dans l'interprétation proposée au TP n°7, le résultat d'une comparaison est comparé au résultat d'un match, tandis que les $z_i$ représentent le niveau de chaque "équipe".

**Q3** Pourquoi cette expression conduit-elle à ce que, après apprentissage, "the estimated
value of positive samples is greater than that of negative samples
for each pair of positive/negative samples" ?

Pour minimiser la loss, le réseau est contraint à maximiser z_i - z_j (resp. z_j - z_i)) lorsque $i$ correspond au "positive sample" (resp. "negative sample"), ie lorsque $y_{ij} = 1$ (resp. $y_{ij} = 0$).

**Q4** Dans le cadre d'une approche par deep learning, quels termes utilise-t-on pour qualifier les réseaux de neurones exploités et la modalité suivant laquelle ils sont entraînés ?

On parle de "ranking functions" ou "rankers". C'est le terme de "siamese networks" qui décrit la modalité selon laquelle ces réseaux sont entraînés.